In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Force CPU execution
device = torch.device("cpu")

# === Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# === ANN definition
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Load data
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values

# === Train-test split & scaling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === Evaluation function
def evaluate_model(h1, h2, h3, lr):
    set_seed(42)
    model = ANN(X_train.shape[1], h1, h2, h3).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_tensor).cpu().numpy()
    return r2_score(y_test, y_pred_test)

# === ACO Initialization
def value(i): return 16 + i
NUM_ANTS = 10
NUM_ITER = 10
pheromone = np.ones((113, 113, 113))
best_score = -np.inf
best_params = None

# === ACO Loop
for gen in range(NUM_ITER):
    print(f"\n🔄 ACO Generation {gen+1}")
    for _ in range(NUM_ANTS):
        h1i = np.random.choice(113, p=pheromone.sum((1,2)) / pheromone.sum())
        h2i = np.random.choice(113, p=pheromone[h1i].sum(1) / pheromone[h1i].sum())
        h3i = np.random.choice(113, p=pheromone[h1i,h2i] / pheromone[h1i,h2i].sum())
        lr = round(random.uniform(0.0001, 0.01), 5)

        h1, h2, h3 = value(h1i), value(h2i), value(h3i)
        score = evaluate_model(h1, h2, h3, lr)

        if score > best_score:
            best_score = score
            best_params = (h1, h2, h3, lr)

        pheromone[h1i, h2i, h3i] += score

    print(f"Top R² so far: {best_score:.6f} | Params: {best_params}")

# === Final model training with best ACO params
print("\n🏁 Final Training with ACO Best Parameters")
final_model = ANN(X_train.shape[1], *best_params[:3]).to(device)
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params[3])
criterion = nn.MSELoss()
set_seed(42)
for epoch in range(100):
    final_model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = final_model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

# === Final evaluation
final_model.eval()
with torch.no_grad():
    y_train_pred = final_model(X_train_tensor).cpu().numpy()
    y_test_pred = final_model(X_test_tensor).cpu().numpy()

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae = mean_absolute_error(y_test, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mae_train = mean_absolute_error(y_train, y_train_pred)

# === Print final results
print(f"\n📊 Final ACO-ANN Evaluation for FoS")
print(f"Best Hyperparameters: {best_params}")
print(f"✅ R² (Train): {r2_train:.6f}")
print(f"✅ R² (Test):  {r2_test:.6f}")
print(f"📉 RMSE (Train): {rmse_train:.6f}")
print(f"📉 RMSE (Test):  {rmse:.6f}")
print(f"📉 MAE (Train): {mae_train:.6f}")
print(f"📉 MAE (Test):  {mae:.6f}")




🔄 ACO Generation 1
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 2
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 3
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 4
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 5
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 6
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 7
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 8
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 9
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 10
Top R² so far: 0.904299 | Params: (58, 123, 98, 0.00643)

🏁 Final Training with ACO Best Parameters

📊 Final ACO-ANN Evaluation for FoS
Best Hyperparameters: (58, 123, 98, 0.00643)
✅ R² (Train): 0.997709
✅ R² (Test):  0.906126
📉 RMSE (Train): 0.055191
📉 RMSE (Test):  0.389779
📉 MAE (Tr

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Force CPU execution
device = torch.device("cpu")

# === Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# === ANN definition
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Load data for SeismicFoS
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["SeismicFoS"].values  # ✅ Target: SeismicFoS

# === Train-test split & scaling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === Model evaluation function
def evaluate_model(h1, h2, h3, lr):
    set_seed(42)
    model = ANN(X_train.shape[1], h1, h2, h3).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_tensor).cpu().numpy()
    return r2_score(y_test, y_pred_test)

# === ACO setup
def value(i): return 16 + i
NUM_ANTS = 10
NUM_ITER = 10
pheromone = np.ones((113, 113, 113))  # For h1, h2, h3 options
best_score = -np.inf
best_params = None

# === ACO optimization loop
for gen in range(NUM_ITER):
    print(f"\n🔄 ACO Generation {gen+1}")
    for _ in range(NUM_ANTS):
        h1i = np.random.choice(113, p=pheromone.sum((1,2)) / pheromone.sum())
        h2i = np.random.choice(113, p=pheromone[h1i].sum(1) / pheromone[h1i].sum())
        h3i = np.random.choice(113, p=pheromone[h1i,h2i] / pheromone[h1i,h2i].sum())
        lr = round(random.uniform(0.0001, 0.01), 5)

        h1, h2, h3 = value(h1i), value(h2i), value(h3i)
        score = evaluate_model(h1, h2, h3, lr)

        if score > best_score:
            best_score = score
            best_params = (h1, h2, h3, lr)

        pheromone[h1i, h2i, h3i] += score

    print(f"Top R² so far: {best_score:.6f} | Params: {best_params}")

# === Final training with best parameters
print("\n🏁 Final Training with ACO Best Parameters")
final_model = ANN(X_train.shape[1], *best_params[:3]).to(device)
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params[3])
criterion = nn.MSELoss()
set_seed(42)

for epoch in range(100):
    final_model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = final_model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

# === Final evaluation
final_model.eval()
with torch.no_grad():
    y_train_pred = final_model(X_train_tensor).cpu().numpy()
    y_test_pred = final_model(X_test_tensor).cpu().numpy()

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# === Print final results
print(f"\n📊 Final ACO-ANN Evaluation for SeismicFoS")
print(f"Best Hyperparameters: {best_params}")
print(f"✅ R² (Train): {r2_train:.6f}")
print(f"✅ R² (Test):  {r2_test:.6f}")
print(f"📉 RMSE (Train): {rmse_train:.6f}")
print(f"📉 RMSE (Test):  {rmse_test:.6f}")
print(f"📉 MAE (Train): {mae_train:.6f}")
print(f"📉 MAE (Test):  {mae_test:.6f}")



🔄 ACO Generation 1
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 2
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 3
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 4
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 5
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 6
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 7
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 8
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 9
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🔄 ACO Generation 10
Top R² so far: 0.787456 | Params: (58, 123, 98, 0.00643)

🏁 Final Training with ACO Best Parameters

📊 Final ACO-ANN Evaluation for SeismicFoS
Best Hyperparameters: (58, 123, 98, 0.00643)
✅ R² (Train): 0.960125
✅ R² (Test):  0.796276
📉 RMSE (Train): 0.174396
📉 RMSE (Test):  0.423792
📉 